# Python Task 2

Question 1: Distance Matrix Calculation
Create a function named calculate_distance_matrix that takes the dataset-3.csv as input and generates a DataFrame representing distances between IDs.

The resulting DataFrame should have cumulative distances along known routes, with diagonal values set to 0. If distances between toll locations A to B and B to C are known, then the distance from A to C should be the sum of these distances. Ensure the matrix is symmetric, accounting for bidirectional distances between toll locations (i.e. A to B is equal to B to A).

In [1]:
import pandas as pd
import networkx as nx

def calculate_distance_matrix(dataset_path):
    # Read the dataset into a DataFrame
    df = pd.read_csv(dataset_path)

    # Create a directed graph
    G = nx.DiGraph()

    # Add edges and distances from the dataset to the graph
    for _, row in df.iterrows():
        G.add_edge(row['id_start'], row['id_end'], distance=row['distance'])
        G.add_edge(row['id_end'], row['id_start'], distance=row['distance'])  # Add reverse edge

    # Calculate the shortest path distances between all pairs of nodes
    all_pairs_distances = dict(nx.all_pairs_dijkstra_path_length(G))

    # Create a DataFrame for the distance matrix
    nodes = list(G.nodes())
    distance_matrix = pd.DataFrame(index=nodes, columns=nodes)

    # Fill the distance matrix with cumulative distances
    for start_node in nodes:
        for end_node in nodes:
            distance_matrix.loc[start_node, end_node] = all_pairs_distances[start_node][end_node]

    return distance_matrix


dataset_path = 'dataset-3.csv'
distance_matrix = calculate_distance_matrix(dataset_path)
print(distance_matrix)




          1001400.0 1001402.0 1001404.0 1001406.0 1001408.0 1001410.0  \
1001400.0         0         1         2         3         4         5   
1001402.0         1         0         1         2         3         4   
1001404.0         2         1         0         1         2         3   
1001406.0         3         2         1         0         1         2   
1001408.0         4         3         2         1         0         1   
1001410.0         5         4         3         2         1         0   
1001412.0         6         5         4         3         2         1   
1001414.0         7         6         5         4         3         2   
1001416.0         8         7         6         5         4         3   
1001418.0         9         8         7         6         5         4   
1001420.0        10         9         8         7         6         5   
1001422.0        11        10         9         8         7         6   
1001424.0        12        11        10         9  

Question 2: Unroll Distance Matrix
Create a function unroll_distance_matrix that takes the DataFrame created in Question 1. The resulting DataFrame should have three columns: columns id_start, id_end, and distance.

All the combinations except for same id_start to id_end must be present in the rows with their distance values from the input DataFrame.

In [2]:
import pandas as pd

def unroll_distance_matrix(distance_matrix):
    # Reset index to convert node names from index to a column
    distance_matrix = distance_matrix.reset_index()

    # Melt the DataFrame to unroll the distance matrix
    unrolled_distances = pd.melt(distance_matrix, id_vars='index', var_name='id_end', value_name='distance')

    # Rename columns for clarity
    unrolled_distances.columns = ['id_start', 'id_end', 'distance']

    # Filter out rows where id_start is equal to id_end
    unrolled_distances = unrolled_distances[unrolled_distances['id_start'] != unrolled_distances['id_end']]

    # Reset index
    unrolled_distances.reset_index(drop=True, inplace=True)

    return unrolled_distances


unrolled_distances = unroll_distance_matrix(distance_matrix)
print(unrolled_distances)


       id_start     id_end distance
0     1001402.0  1001400.0        1
1     1001404.0  1001400.0        2
2     1001406.0  1001400.0        3
3     1001408.0  1001400.0        4
4     1001410.0  1001400.0        5
...         ...        ...      ...
1801  1001462.0  1001472.0        5
1802  1001464.0  1001472.0        4
1803  1001466.0  1001472.0        3
1804  1001468.0  1001472.0        2
1805  1001470.0  1001472.0        1

[1806 rows x 3 columns]


Question 3: Finding IDs within Percentage Threshold
Create a function find_ids_within_ten_percentage_threshold that takes the DataFrame created in Question 2 and a reference value from the id_start column as an integer.

Calculate average distance for the reference value given as an input and return a sorted list of values from id_start column which lie within 10% (including ceiling and floor) of the reference value's average.

In [3]:
def find_ids_within_ten_percentage_threshold(unrolled_distances, reference_value):
    # Filter rows where id_start is equal to the reference value
    reference_rows = unrolled_distances[unrolled_distances['id_start'] == reference_value]

    # Calculate the average distance for the reference value
    average_distance = reference_rows['distance'].mean()

    # Calculate the lower and upper bounds within 10% of the average distance
    lower_bound = average_distance - (average_distance * 0.10)
    upper_bound = average_distance + (average_distance * 0.10)

    # Filter rows where distance is within the 10% threshold
    result = unrolled_distances[(unrolled_distances['distance'] >= lower_bound) & (unrolled_distances['distance'] <= upper_bound)]

    # Extract and sort unique values from id_start column
    result_ids = sorted(result['id_start'].unique())

    return result_ids


reference_value = 1  
result_ids = find_ids_within_ten_percentage_threshold(unrolled_distances, reference_value)
print(result_ids)



[]


Question 4: Calculate Toll Rate
Create a function calculate_toll_rate that takes the DataFrame created in Question 2 as input and calculates toll rates based on vehicle types.

The resulting DataFrame should add 5 columns to the input DataFrame: moto, car, rv, bus, and truck with their respective rate coefficients. The toll rates should be calculated by multiplying the distance with the given rate coefficients for each vehicle type:

0.8 for moto
1.2 for car
1.5 for rv
2.2 for bus
3.6 for truck

In [4]:
def calculate_toll_rate(unrolled_distances):
    # Define rate coefficients for each vehicle type
    rate_coefficients = {'moto': 0.8, 'car': 1.2, 'rv': 1.5, 'bus': 2.2, 'truck': 3.6}

    # Calculate toll rates for each vehicle type
    for vehicle_type, rate in rate_coefficients.items():
        unrolled_distances[vehicle_type] = unrolled_distances['distance'] * rate

    return unrolled_distances


toll_rates_df = calculate_toll_rate(unrolled_distances)
print(toll_rates_df)


       id_start     id_end distance moto  car   rv   bus truck
0     1001402.0  1001400.0        1  0.8  1.2  1.5   2.2   3.6
1     1001404.0  1001400.0        2  1.6  2.4  3.0   4.4   7.2
2     1001406.0  1001400.0        3  2.4  3.6  4.5   6.6  10.8
3     1001408.0  1001400.0        4  3.2  4.8  6.0   8.8  14.4
4     1001410.0  1001400.0        5  4.0  6.0  7.5  11.0  18.0
...         ...        ...      ...  ...  ...  ...   ...   ...
1801  1001462.0  1001472.0        5  4.0  6.0  7.5  11.0  18.0
1802  1001464.0  1001472.0        4  3.2  4.8  6.0   8.8  14.4
1803  1001466.0  1001472.0        3  2.4  3.6  4.5   6.6  10.8
1804  1001468.0  1001472.0        2  1.6  2.4  3.0   4.4   7.2
1805  1001470.0  1001472.0        1  0.8  1.2  1.5   2.2   3.6

[1806 rows x 8 columns]


Question 5: Calculate Time-Based Toll Rates
Create a function named calculate_time_based_toll_rates that takes the DataFrame created in Question 3 as input and calculates toll rates for different time intervals within a day.

The resulting DataFrame should have these five columns added to the input: start_day, start_time, end_day, and end_time.

start_day, end_day must be strings with day values (from Monday to Sunday in proper case)
start_time and end_time must be of type datetime.time() with the values from time range given below.
Modify the values of vehicle columns according to the following time ranges:

Weekdays (Monday - Friday):

From 00:00:00 to 10:00:00: Apply a discount factor of 0.8
From 10:00:00 to 18:00:00: Apply a discount factor of 1.2
From 18:00:00 to 23:59:59: Apply a discount factor of 0.8
Weekends (Saturday and Sunday):

Apply a constant discount factor of 0.7 for all times.
For each unique (id_start, id_end) pair, cover a full 24-hour period (from 12:00:00 AM to 11:59:59 PM) and span all 7 days of the week (from Monday to Sunday).

In [20]:
import pandas as pd
from datetime import time

def calculate_time_based_toll_rates(df):
    # Define time ranges and corresponding discount factors
    time_ranges = [
        (time(0, 0, 0), time(10, 0, 0), 0.8),    # Weekdays: 00:00:00 to 10:00:00
        (time(10, 0, 0), time(18, 0, 0), 1.2),   # Weekdays: 10:00:00 to 18:00:00
        (time(18, 0, 0), time(23, 59, 59), 0.8), # Weekdays: 18:00:00 to 23:59:59
        (time(0, 0, 0), time(23, 59, 59), 0.7)   # Weekends: 00:00:00 to 23:59:59
    ]

    # Convert 'start_time' and 'end_time' columns to datetime.time type
    df['start_time'] = pd.to_datetime(df['start_time']).dt.time
    df['end_time'] = pd.to_datetime(df['end_time']).dt.time

    # Iterate over each time range and apply discount factors
    for start_time, end_time, discount_factor in time_ranges:
        mask = (df['start_time'] >= start_time) & (df['end_time'] <= end_time)
        df.loc[mask, ['moto', 'car', 'rv', 'bus', 'truck']] *= discount_factor

    return df

# Sample DataFrame
sample_df = pd.DataFrame({
    'id_start': ['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', '1001400', '1001400', '1001400'],
    'id_end': ['B', 'B', 'B', 'B', 'C', 'C', 'C', 'C', '1001402', '1001402', '1001402'],
    'distance': [10, 10, 10, 10, 15, 15, 15, 15, 9.7, 9.7, 9.7],
    'start_day': ['Monday', 'Tuesday', 'Saturday', 'Sunday', 'Friday', 'Saturday', 'Sunday', 'Sunday', 'Wednesday', 'Saturday', 'Monday'],
    'start_time': ['00:00:00', '10:00:00', '18:00:00', '23:59:59', '10:00:00', '18:00:00', '23:59:59', '23:59:59', '11:00:00', '18:00:00', '11:00:00'],
    'end_day': ['Friday', 'Saturday', 'Sunday', 'Sunday', 'Saturday', 'Sunday', 'Sunday', 'Wednesday', 'Saturday', 'Monday', 'Tuesday'],
    'end_time': ['10:00:00', '18:00:00', '23:59:59', '23:59:59', '18:00:00', '23:59:59', '23:59:59', '11:00:00', '18:00:00', '11:00:00', '11:00:00'],
    'moto': [6.21, 9.31, 6.21, 5.43, 7.10, 10.66, 7.10, 6.22, 11.1, 11.1, 11.1],
    'car': [9.31, 13.97, 9.31, 8.15, 10.66, 15.98, 10.66, 9.32, 16.65, 16.65, 16.65],
    'rv': [11.64, 17.46, 11.64, 10.19, 13.32, 19.98, 13.32, 11.66, 20.81, 20.81, 20.81],
    'bus': [17.07, 25.61, 17.07, 14.94, 19.54, 29.30, 19.54, 17.09, 30.58, 30.58, 30.58],
    'truck': [27.94, 41.90, 27.94, 24.44, 31.97, 47.95, 31.97, 27.97, 50.13, 50.13, 50.13]
})

# Apply the function to the sample DataFrame
result_df = calculate_time_based_toll_rates(sample_df)


print(result_df)




   id_start   id_end  distance  start_day start_time    end_day  end_time  \
0         A        B      10.0     Monday   00:00:00     Friday  10:00:00   
1         A        B      10.0    Tuesday   10:00:00   Saturday  18:00:00   
2         A        B      10.0   Saturday   18:00:00     Sunday  23:59:59   
3         A        B      10.0     Sunday   23:59:59     Sunday  23:59:59   
4         A        C      15.0     Friday   10:00:00   Saturday  18:00:00   
5         A        C      15.0   Saturday   18:00:00     Sunday  23:59:59   
6         A        C      15.0     Sunday   23:59:59     Sunday  23:59:59   
7         A        C      15.0     Sunday   23:59:59  Wednesday  11:00:00   
8   1001400  1001402       9.7  Wednesday   11:00:00   Saturday  18:00:00   
9   1001400  1001402       9.7   Saturday   18:00:00     Monday  11:00:00   
10  1001400  1001402       9.7     Monday   11:00:00    Tuesday  11:00:00   

       moto       car        rv       bus     truck  
0   3.47760   5.21360